In [1]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml
import pandas as pd

from lib.notebooks import plot_groups_metrics
from synthesizer import Synthesizer

/tmp/ipykernel_859/3534759458.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
synthesizers_path = glob("../out/synthesizer/*/")
synthesizers_path.sort()

In [3]:
groups_metrics = {}
synthesizers_loss = {
    "path": [],
    "dataset": [],
    "hidden_layers": [],
    "dropout_p": [],
    "learning_rate": [],
    "final_loss": [],
}

for synthesizer_path in synthesizers_path:
    synthesizer = Synthesizer.reload(synthesizer_path, load_nn=False)
    config = synthesizer.config
    with open("%s/metrics.pickle" % synthesizer_path, "rb") as f:
        metrics = pickle.load(f)
    final_loss = min(metrics["validation"]["total"])
    
    synthesizers_loss["path"].append(synthesizer_path[-5:])
    synthesizers_loss["dataset"].append(config['dataset']['name'])
    synthesizers_loss["hidden_layers"].append(f"{len(config['model']['hidden_layers'])}x{config['model']['hidden_layers'][0]}")
    synthesizers_loss["dropout_p"].append(config['model']['dropout_p'])
    synthesizers_loss["learning_rate"].append(config['training']['learning_rate'])
    synthesizers_loss["final_loss"].append(final_loss)
    
    group_name = "\n".join((
        f"{config['dataset']['name']}",
        f"hidden_layers={len(config['model']['hidden_layers'])}x{config['model']['hidden_layers'][0]}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][synthesizer_path] = metrics

synthesizers_loss = pd.DataFrame(synthesizers_loss)

In [6]:
synthesizers_loss.sort_values("final_loss").head(20)
#print(synthesizers_loss['path'][0])

,path,dataset,hidden_layers,dropout_p,learning_rate,final_loss
0,9d-0/,pb2007,4x512,0.05,0.0005,0.921502
2,9d-2/,pb2007,4x512,0.05,0.0005,0.924421
4,9d-4/,pb2007,4x512,0.05,0.0005,0.925878
1,9d-1/,pb2007,4x512,0.05,0.0005,0.928283
3,9d-3/,pb2007,4x512,0.05,0.0005,0.939316


In [31]:
print(groups_metrics['pb2007\nhidden_layers=4x512']['../out/synthesizer/ea587b76c95fecef01cfd16c7f5f289d-0/']['train']['total'][-1],
      groups_metrics['pb2007\nhidden_layers=4x512']['../out/synthesizer/ea587b76c95fecef01cfd16c7f5f289d-1/']['train']['total'][-1],
      groups_metrics['pb2007\nhidden_layers=4x512']['../out/synthesizer/ea587b76c95fecef01cfd16c7f5f289d-2/']['train']['total'][-1],
      groups_metrics['pb2007\nhidden_layers=4x512']['../out/synthesizer/ea587b76c95fecef01cfd16c7f5f289d-3/']['train']['total'][-1],
      groups_metrics['pb2007\nhidden_layers=4x512']['../out/synthesizer/ea587b76c95fecef01cfd16c7f5f289d-4/']['train']['total'][-1]
      )
#'../out/synthesizer/ea587b76c95fecef01cfd16c7f5f289d-0/'][


#print(groups_metrics.keys())


0.921588835415539 0.9211917272559161 0.9227776135410276 0.9227813643378182 0.9212868514361685


In [15]:
df = pd.DataFrame(groups_metrics)


InvalidIndexError: (slice(None, None, None), 0)

In [7]:
metrics_name = [
    "total",
]

def show_metrics(split_name="validation"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation"]))

interactive(children=(Dropdown(description='split_name', index=1, options=('train', 'validation'), value='vali…